### Logistic Regression
- Estimates the probability of an event occurring based on independent variables
- Models a binary outcome (True/False, Yes/No)
    - In this example, the outcome is either win or loss
- Equation: $$ f(x) = {e^{b} \over (1+e^{-x})} $$

### Problem Statement and Hypothesis
- Research Question: Can the business predict opportunity outcomes?
- Alternate Hypothesis: Yes, we can predict opportunity outcomes
- Null Hypothesis: No, we cannot predict opportunity outcomes

### Why is it important to predict an opportunity with statistical significance?
- Anticipate sales for CST and other leadership
- Help influence a positive behavior


**Expected Outcome:** To have a model that can generalize trends in data using significant variables to predict a win or a loss.
- Understand which opportunity variables are most significant
- Figure out the accuracy of the model

**Data Collection Concerns:**
- Dataset needs to have at least 10,000 records to prevent bias
- At least 10 variables 
- At least 1 continuous and 1 categorical variable
- Rights to use the data

**Data Variables:**
- **Independent (x<sub>1</sub>-x<sub>n</sub>):** All other variables
- **Dependent (Y):** Stage (Categorical variable - Outcomes are only win or loss)
    - Logistic regressions are used to predict categorical outcomes

**Packages:**
- **pandas:** Used for dataframes
- **pyodbc:** Used to connect to SQL Server
- **sklearn/stats model API:** Used for our logistic model
    - Create train and test sets
    - Recursive feature elimination
    - Accuracy/precision/matrix scores
    - Receiver operator characteristic
- **seaborn/matplotlib:** Used for visualization

**Preprocessing Technique:**
- Import packages and load data
- Review the data
    - head(), info()
    - Handle missingness, outliers
- Exploratory analysis
- Variable selection

### Import Packages and load data

In [165]:
#import packages
import pyodbc as db
import pandas as pd

#connect to DB
server = 'tcp:usaepwvds025.database.windows.net' 
database = 'SIMS' 
username = 'ciaserviceaccount' 
password = 'pipeline$330' 
cnxn = db.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

#load dataframe
sql = "select * from sims.logisticRegression"
df = pd.read_sql(sql,cnxn)


C:\Users\EHIDRPC\AppData\Local\Temp\ipykernel_42728\1958964111.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql,cnxn)


### Review the data

In [166]:
df.head()

,LegacySFInstance,LegacyAccountID,LegacyAccountName,CurrentCompanyGroup,CurrentCompanyID,CurrentCompanyMarketCap,CurrentCompanyMarketCapCategory,Top12CMMCompany,Top24CMMCompany,OpportunityID,OpportunityName,OpportunityType,TotalOpportunityAmount,CreateDate,StartDate,OpportunityOwner,StageName,CloseReason,CloseComments,ImpactedFY
0,RxC,0013s00000uY8t8AAC,"Coram Speciality Infusion Services, Inc.","Coram Speciality Infusion Services, Inc.",NaN,NaN,None,None,None,0063s00000DSJH9AAP,Coram/CVS Copay Finder,New,NaN,2019-08-06 19:15:55,None,Andrew Burman,Closed Lost,No Bid,Nothing materialized with the client.,FY 2020
1,RxC,0013s00000xIwQOAA0,Caris-MPI,Caris-MPI,NaN,NaN,None,None,None,0063s00000EAht5AAD,Precision Medicine Survey,New,200000.0,2017-05-31 15:15:14,None,Jennifer Frytak,Closed Lost,Client Funding Cut,None,FY 2018
2,RxC,0013s00000zTk3cAAC,Pharma Pac,Pharma Pac,NaN,NaN,None,None,None,0063s00000DUuUEAA1,Pharma Pac,New,173838.0,2020-04-09 15:52:13,2021-04-12,Ryan Grimmett,Closed Lost,Cancelled,None,FY 2022
3,RxC,0013s000011ZefwAAC,Biologics Financial Reconciliation,Biologics Financial Reconciliation,NaN,NaN,None,None,None,0063s00000EwxCLAAZ,Biologics RARE - Launched for FY22 Tracking,Renewal,44986789.0,2021-02-04 14:55:41,2021-04-01,Laura Sieve,Closed Won,None,None,FY 2022
4,RxC,0013s000011ZefwAAC,Biologics Financial Reconciliation,Biologics Financial Reconciliation,NaN,NaN,None,None,None,0063s00000ExCSbAAN,Biologics ONC - Launched for FY22 Tracking,Renewal,18086000.0,2021-04-27 15:01:37,2021-04-01,Laura Sieve,Closed Won,None,None,FY 2022


- Investigate missingness
- Understand data types (Continuous vs Categorical)
- Scale
- Outliers

In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27493 entries, 0 to 27492
Data columns (total 20 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   LegacySFInstance                 27493 non-null  object        
 1   LegacyAccountID                  27493 non-null  object        
 2   LegacyAccountName                27473 non-null  object        
 3   CurrentCompanyGroup              27473 non-null  object        
 4   CurrentCompanyID                 27416 non-null  float64       
 5   CurrentCompanyMarketCap          23047 non-null  float64       
 6   CurrentCompanyMarketCapCategory  27416 non-null  object        
 7   Top12CMMCompany                  9975 non-null   object        
 8   Top24CMMCompany                  4445 non-null   object        
 9   OpportunityID                    27493 non-null  object        
 10  OpportunityName                  27493 non-null  object   

#### Drop ID Columns
- Looking for patterns in the data. Since the ID columns are unique, this won't help us in predicting our win/loss opportunities

In [167]:
df = df.drop(columns=['LegacyAccountID', 'CurrentCompanyID', 'OpportunityID'])

We can't impute comments, so we will drop `CloseComments` from our dataset

In [168]:
df = df.drop(columns = 'CloseComments')

Since we have `CreateDate` that have no null values, we will use this column and drop `StartDate`

In [169]:
df = df.drop(columns = 'StartDate')

In [170]:
df.query('LegacyAccountName != LegacyAccountName')

,LegacySFInstance,LegacyAccountName,CurrentCompanyGroup,CurrentCompanyMarketCap,CurrentCompanyMarketCapCategory,Top12CMMCompany,Top24CMMCompany,OpportunityName,OpportunityType,TotalOpportunityAmount,CreateDate,OpportunityOwner,StageName,CloseReason,ImpactedFY
57,CMM,None,None,NaN,None,None,None,MDD Xadago (Safinamide) Prior Authorization Pl...,Renewal,105250.00,2020-03-02 15:53:43,Amanda Tegeder,Closed Won,None,FY 2022
58,CMM,None,None,NaN,None,None,None,MDD Xadago (Safinamide) Prior Authorization Pl...,Renewal,105125.00,2021-05-10 17:42:46,David Prevo,Closed Won,None,FY 2023
59,CMM,None,None,NaN,None,None,None,MDD Xadago (Safinamide) HubExpress,New,148435.00,2021-04-19 14:40:04,Josh Lobs,Closed Lost,Budget,FY 2022
60,CMM,None,None,NaN,None,None,None,MDD Xadago (Safinamide) Prior Authorization Pl...,Renewal,90575.00,2022-05-26 16:39:30,David Prevo,Closed Won,None,FY 2024
61,CMM,None,None,NaN,None,None,None,testpharma Brand 1 Test Prior Authorization Pl...,Renewal,50010.00,2019-01-03 21:30:43,Julie Fielding,Closed Lost,None,FY 2019
62,CMM,None,None,NaN,None,None,None,testpharma Brand4 IAN,New,350000.00,2020-10-30 13:44:15,Julie Fielding,Closed Lost,No Bid,FY 2021
63,CMM,None,None,NaN,None,None,None,testpharma Brand 1 Test Provider Medication Al...,New,15.00,2021-02-02 16:27:41,Julie Fielding,Closed Lost,No Bid,FY 2021
64,CMM,None,None,NaN,None,None,None,testpharma BrandABC Prior Authorization Plus,New,1740232.24,2021-10-29 14:21:29,Jason Kemble,Closed Lost,No Bid,FY 2022
65,CMM,None,None,NaN,None,None,None,testpharma BB TEST BRAND Prior Authorization Plus,New,198042.50,2021-11-30 19:44:20,Caroline Savoy,Closed Lost,No Bid,FY 2022
66,CMM,None,None,NaN,None,None,None,testpharma BB TEST BRAND Prior Authorization Plus,New,174042.50,2021-11-30 20:01:53,Caroline Savoy,Closed Lost,No Bid,FY 2022


In [171]:
df.dropna(subset=['LegacyAccountName'], inplace=True)

Impute a value for missing market caps
- Average value in the market cap category
- Some companies don't have a market cap category, so we will first impute market cap categories

In [172]:
modeMarketCap = df['CurrentCompanyMarketCapCategory'].mode()
df['CurrentCompanyMarketCapCategory'].fillna(df['CurrentCompanyMarketCapCategory'].mode()[0], inplace=True)

In [173]:
meanMarketCap = df.groupby('CurrentCompanyMarketCapCategory')['CurrentCompanyMarketCap'].mean().reset_index()
df['CurrentCompanyMarketCap'] = df['CurrentCompanyMarketCap'].fillna(df.groupby('CurrentCompanyMarketCapCategory')['CurrentCompanyMarketCap'].transform('mean'))

df['CurrentCompanyMarketCap'] = df['CurrentCompanyMarketCap'].fillna(df.groupby(['CurrentCompanyMarketCapCategory'])['CurrentCompanyMarketCap'].transform('median'))

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27473 entries, 0 to 27492
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   LegacySFInstance                 27473 non-null  object        
 1   LegacyAccountName                27473 non-null  object        
 2   CurrentCompanyGroup              27473 non-null  object        
 3   CurrentCompanyMarketCap          23104 non-null  float64       
 4   CurrentCompanyMarketCapCategory  27473 non-null  object        
 5   Top12CMMCompany                  9975 non-null   object        
 6   Top24CMMCompany                  4445 non-null   object        
 7   OpportunityName                  27473 non-null  object        
 8   OpportunityType                  27473 non-null  object        
 9   TotalOpportunityAmount           26152 non-null  float64       
 10  CreateDate                       27473 non-null  datetime64[ns]

In [174]:
df['Top12CMMCompany'].fillna('False', inplace = True)
df['Top24CMMCompany'].fillna('False', inplace = True)

In [140]:
df.query('CurrentCompanyMarketCap != CurrentCompanyMarketCap')

,LegacySFInstance,LegacyAccountName,CurrentCompanyGroup,CurrentCompanyMarketCap,CurrentCompanyMarketCapCategory,Top12CMMCompany,Top24CMMCompany,OpportunityName,OpportunityType,TotalOpportunityAmount,CreateDate,OpportunityOwner,StageName,CloseReason,ImpactedFY
92,RxC,"Affymax, Inc",Affymax,NaN,Not Available,False,False,Onmontys REMS Services,New,1500000.0,2013-04-10 18:10:37,Kelly Wygal,Closed Lost,McKesson Disqualified,FY 2014
93,RHP,Apotex,Apotex,NaN,Private,False,False,Apotex_MSA+eV Master SOW,New,0.0,2021-08-03 04:17:21,Laura Redman,Closed Lost,No Bid,FY 2024
94,RxC,Apotex,Apotex,NaN,Private,False,False,2015 Focus Group,New,48750.0,2015-01-21 18:12:14,John Archie,Closed Lost,Client Funding Cut,FY 2015
95,RxC,Apotex,Apotex,NaN,Private,False,False,ApoBiologix,New,50000.0,2015-01-22 06:00:00,Dana Mahoney,Closed Lost,None,FY 2015
96,RxC,Aradigm Corporation,Aradigm,NaN,Not Available,False,False,Aradigm- Pulmaquin,New,0.0,2018-09-17 22:26:58,Andrew Burman,Closed Lost,No Bid,FY 2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27488,RHP,Biocon Biologics Inc.,Biocon Biologics Inc.,NaN,Not Available,False,False,Biocon Biologics_TS SOW,New,62980.0,2023-06-14 21:23:02,Laura Redman,Closed Won,None,FY 2024
27489,RHP,Biocon Biologics Inc.,Biocon Biologics Inc.,NaN,Not Available,False,False,"Biocon Biologics_State Insulin Prog (MN, CO, ME)",New,117520.0,2023-06-29 23:19:29,Laura Redman,Closed Won,None,FY 2024
27490,RHP,Biocon Biologics Inc.,Biocon Biologics Inc.,NaN,Not Available,False,False,"Biocon Biologics_LS, TS & State Insulin Imp SOW",New,0.0,2023-06-29 23:25:26,Laura Redman,Closed Won,None,FY 2024
27491,RHP,Biocon Biologics Inc.,Biocon Biologics Inc.,NaN,Not Available,False,False,"Biocon Biologics_State Insulin Prog (MN, CO, M...",New,0.0,2024-01-15 17:36:03,Laura Redman,Closed Lost,No Bid,FY 2024


In [175]:
df['CreateDateMonth'] = df['CreateDate'].dt.month 

In [176]:
df['TotalOpportunityAmount'].fillna(0.00, inplace = True)

In [182]:
# df.loc[df.CloseReason == "Closed Won", "CloseReason"] = "Won"
# df.loc[(df['CloseReason'].isna()) & (df['StageName'] == "Closed Won"), 'CloseReason'] = "Won"

CloseReasonMode = df['CloseReason'].mode()
print(CloseReasonMode)

df.loc[(df['CloseReason'].isna()) & (df['StageName'] == "Closed Lost"), 'CloseReason'] = "CloseReasonMode"

df.loc[(df['CloseReason'].isna()) & (df['StageName'] == "Closed Won"), 'CloseReason'] = "Won"


0    No Bid
Name: CloseReason, dtype: object


In [183]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27473 entries, 0 to 27492
Data columns (total 16 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   LegacySFInstance                 27473 non-null  object        
 1   LegacyAccountName                27473 non-null  object        
 2   CurrentCompanyGroup              27473 non-null  object        
 3   CurrentCompanyMarketCap          23104 non-null  float64       
 4   CurrentCompanyMarketCapCategory  27473 non-null  object        
 5   Top12CMMCompany                  27473 non-null  object        
 6   Top24CMMCompany                  27473 non-null  object        
 7   OpportunityName                  27473 non-null  object        
 8   OpportunityType                  27473 non-null  object        
 9   TotalOpportunityAmount           27473 non-null  float64       
 10  CreateDate                       27473 non-null  datetime64[ns]

In [184]:
display(df)

,LegacySFInstance,LegacyAccountName,CurrentCompanyGroup,CurrentCompanyMarketCap,CurrentCompanyMarketCapCategory,Top12CMMCompany,Top24CMMCompany,OpportunityName,OpportunityType,TotalOpportunityAmount,CreateDate,OpportunityOwner,StageName,CloseReason,ImpactedFY,CreateDateMonth
0,RxC,"Coram Speciality Infusion Services, Inc.","Coram Speciality Infusion Services, Inc.",8.330270e+10,Large,False,False,Coram/CVS Copay Finder,New,0.0,2019-08-06 19:15:55,Andrew Burman,Closed Lost,No Bid,FY 2020,8
1,RxC,Caris-MPI,Caris-MPI,8.330270e+10,Large,False,False,Precision Medicine Survey,New,200000.0,2017-05-31 15:15:14,Jennifer Frytak,Closed Lost,Client Funding Cut,FY 2018,5
2,RxC,Pharma Pac,Pharma Pac,8.330270e+10,Large,False,False,Pharma Pac,New,173838.0,2020-04-09 15:52:13,Ryan Grimmett,Closed Lost,Cancelled,FY 2022,4
3,RxC,Biologics Financial Reconciliation,Biologics Financial Reconciliation,8.330270e+10,Large,False,False,Biologics RARE - Launched for FY22 Tracking,Renewal,44986789.0,2021-02-04 14:55:41,Laura Sieve,Closed Won,Won,FY 2022,2
4,RxC,Biologics Financial Reconciliation,Biologics Financial Reconciliation,8.330270e+10,Large,False,False,Biologics ONC - Launched for FY22 Tracking,Renewal,18086000.0,2021-04-27 15:01:37,Laura Sieve,Closed Won,Won,FY 2022,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27488,RHP,Biocon Biologics Inc.,Biocon Biologics Inc.,NaN,Not Available,False,False,Biocon Biologics_TS SOW,New,62980.0,2023-06-14 21:23:02,Laura Redman,Closed Won,Won,FY 2024,6
27489,RHP,Biocon Biologics Inc.,Biocon Biologics Inc.,NaN,Not Available,False,False,"Biocon Biologics_State Insulin Prog (MN, CO, ME)",New,117520.0,2023-06-29 23:19:29,Laura Redman,Closed Won,Won,FY 2024,6
27490,RHP,Biocon Biologics Inc.,Biocon Biologics Inc.,NaN,Not Available,False,False,"Biocon Biologics_LS, TS & State Insulin Imp SOW",New,0.0,2023-06-29 23:25:26,Laura Redman,Closed Won,Won,FY 2024,6
27491,RHP,Biocon Biologics Inc.,Biocon Biologics Inc.,NaN,Not Available,False,False,"Biocon Biologics_State Insulin Prog (MN, CO, M...",New,0.0,2024-01-15 17:36:03,Laura Redman,Closed Lost,No Bid,FY 2024,1
